In [1]:
flexzone_url = "https://api.nextbike.net/reservation/geojson/flexzone_bn.json"

# call the API to get the Flexzone data
import requests
response = requests.get(flexzone_url)

# check if the request was successful
if response.status_code == 200:
    flexzone_data = response.json()
    print("Flexzone data retrieved successfully.")
    print(flexzone_data)
else:
    print(f"Failed to retrieve Flexzone data. Status code: {response.status_code}")
    flexzone_data = None


print("dsfkdsfkaöksdfölk")

Flexzone data retrieved successfully.
{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[[13.374238962785, 52.528078809549], [13.375752, 52.528559], [13.37216, 52.532239], [13.37167, 52.53307], [13.367679, 52.536595], [13.367776, 52.536634], [13.368576, 52.535954], [13.373087, 52.538079], [13.371594, 52.539017], [13.369931, 52.538187], [13.369854, 52.537932], [13.369121, 52.537589], [13.369094, 52.537748], [13.369698, 52.538071], [13.369799, 52.538345], [13.371371, 52.53913], [13.370356, 52.539674], [13.366471, 52.537599], [13.366466, 52.537778], [13.368724, 52.539011], [13.369965, 52.539977], [13.368472, 52.540908], [13.361638, 52.537833], [13.359995, 52.539013], [13.360124, 52.53911], [13.360561, 52.538809], [13.361148, 52.539104], [13.361207, 52.538936], [13.360745, 52.538681], [13.361615, 52.538145], [13.368096, 52.541001], [13.368127, 52.541175], [13.366469, 52.54214], [13.366479, 52.542318], [13.368257, 52.541331], [13.

In [2]:
import pandas as pd
import numpy as np
import requests
from shapely.geometry import Point, Polygon
from geopy.distance import geodesic
import matplotlib.pyplot as plt

# 1. Trips aus CSV laden und bereinigen
trip_sheet = '/DATA/TripAnalysis/nextbike_trips.csv'

trip_df = (
    pd.read_csv(trip_sheet.lstrip('/'))
    .pipe(lambda d: d.rename(columns={c: c.strip() for c in d.columns}))
    .assign(**{
        'Duration-Minutes': lambda d: pd.to_numeric(d['Duration-Minutes'], errors='coerce'),
        'Rental-Lat': lambda d: pd.to_numeric(d['Rental-Lat'], errors='coerce'),
        'Rental-Lng': lambda d: pd.to_numeric(d['Rental-Lng'], errors='coerce'),
        'Return-Lat': lambda d: pd.to_numeric(d['Return-Lat'], errors='coerce'),
        'Return-Lng': lambda d: pd.to_numeric(d['Return-Lng'], errors='coerce'),
    })
    .loc[lambda d: ~d.apply(lambda r: r.astype(str).str.contains('unbekannt', case=False).any(), axis=1)]
    .loc[lambda d: d['Duration-Minutes'] > 2]
    .loc[lambda d: ~((d['Rental-Lat'] == d['Return-Lat']) & (d['Rental-Lng'] == d['Return-Lng']))]
    .reset_index(drop=True)
)

print(f"Trips loaded: {len(trip_df)}")

# 2. Nextbike API Daten abrufen (Berlin)
API_URL = "https://api.nextbike.net/maps/nextbike-live.json?city=362"
response = requests.get(API_URL)
data = response.json()

# 3. Stationen aus API extrahieren
station_positions = []
for country in data.get('countries', []):
    for city in country.get('cities', []):
        if city.get('uid') == 362:
            for place in city.get('places', []):
                if place.get('spot', False):  # Stationen haben 'spot'==True
                    lat = place.get('lat')
                    lng = place.get('lng')
                    if lat is not None and lng is not None:
                        station_positions.append((lat, lng))

print(f"Extracted {len(station_positions)} stations from API")

# 4. Flexzonen per URL laden
flexzone_url = "https://api.nextbike.net/reservation/geojson/flexzone_bn.json"
response = requests.get(flexzone_url)
flex_data = response.json()

flex_polygons_c0 = []  # Flexzone
flex_polygons_c1 = []  # Verbotszone

for feature in flex_data.get('features', []):
    geom_type = feature['geometry']['type']
    coords = feature['geometry']['coordinates']
    poly = None
    if geom_type == 'Polygon':
        poly = Polygon([(c[0], c[1]) for c in coords[0]])
    elif geom_type == 'MultiPolygon':
        # Nimm nur das erste Polygon als Beispiel
        poly = Polygon([(c[0], c[1]) for c in coords[0][0]])
    # Unterscheide c0 und c1 anhand 'properties' oder Reihenfolge
    # Hier nehmen wir an: erstes Feature = c0 (Flexzone), zweites = c1 (Verbotszone)
    if feature.get('properties', {}).get('c0', False) or len(flex_polygons_c0) == 0:
        flex_polygons_c0.append(poly)
    else:
        flex_polygons_c1.append(poly)

print(f"Loaded {len(flex_polygons_c0)} Flexzone polygons (c0)")
print(f"Loaded {len(flex_polygons_c1)} Verbotszone polygons (c1)")

# 5. Hilfsfunktionen

def point_in_any_polygon(point, polygons):
    return any(poly.contains(point) for poly in polygons)

def distance_to_nearest_station(lat, lng):
    min_dist = float('inf')
    for s_lat, s_lng in station_positions:
        dist = geodesic((lat, lng), (s_lat, s_lng)).meters
        if dist < min_dist:
            min_dist = dist
    return min_dist

def distance_to_nearest_flexzone(lat, lng):
    point = Point(lng, lat)
    min_dist = float('inf')
    for poly in flex_polygons_c0:
        centroid = poly.centroid
        dist = geodesic((lat, lng), (centroid.y, centroid.x)).meters
        if dist < min_dist:
            min_dist = dist
    return min_dist

def classify_return_zone(lat, lng):
    point = Point(lng, lat)
    if point_in_any_polygon(point, flex_polygons_c0):
        return "Flexzone"
    elif point_in_any_polygon(point, flex_polygons_c1):
        return "Verbotszone"
    else:
        return "NoFlexzone"

# 6. Analyse & Distanzberechnung

trip_df['zone_classification'] = trip_df.apply(
    lambda row: classify_return_zone(row['Return-Lat'], row['Return-Lng']), axis=1
)

trip_df['dist_to_station_m'] = np.nan
trip_df['dist_to_flexzone_m'] = np.nan

for idx, row in trip_df.iterrows():
    lat = row['Return-Lat']
    lng = row['Return-Lng']
    zone = row['zone_classification']
    if zone == "Flexzone":
        trip_df.at[idx, 'dist_to_station_m'] = distance_to_nearest_station(lat, lng)
    elif zone == "NoFlexzone":
        trip_df.at[idx, 'dist_to_station_m'] = distance_to_nearest_station(lat, lng)
        trip_df.at[idx, 'dist_to_flexzone_m'] = distance_to_nearest_flexzone(lat, lng)
    elif zone == "Verbotszone":
        trip_df.at[idx, 'dist_to_station_m'] = distance_to_nearest_station(lat, lng)

print(trip_df[['Bike-Number', 'Return-Location', 'zone_classification', 'dist_to_station_m', 'dist_to_flexzone_m']].head())

# 7. Visualisierung

plt.figure(figsize=(10,6))
for zone in ["Flexzone", "NoFlexzone", "Verbotszone"]:
    subset = trip_df[trip_df['zone_classification'] == zone]
    distances = subset['dist_to_station_m'].dropna()
    if len(distances) == 0:
        continue
    bins = np.arange(0, distances.max() + 500, 500)
    hist, bin_edges = np.histogram(distances, bins=bins)
    plt.plot(bin_edges[:-1], hist, drawstyle='steps-post', label=f'{zone} (n={len(distances)})')

plt.xlabel('Entfernung zur nächsten Station (Meter)')
plt.ylabel('Anzahl der Trips')
plt.title('Verteilung der Trip-Endentfernungen zur nächsten Station nach Zone')
plt.legend()
plt.grid(True)
plt.show()


Trips loaded: 167799
Extracted 2630 stations from API
Loaded 1 Flexzone polygons (c0)
Loaded 15 Verbotszone polygons (c1)


KeyboardInterrupt: 

In [ ]:
import requests
import pandas as pd

# Nextbike API für Berlin
API_URL = "https://api.nextbike.net/maps/nextbike-live.json?city=362"

# API-Daten abrufen
response = requests.get(API_URL)
data = response.json()

# Alle Stationen extrahieren (places mit spot == True)
stations_data = []
for country in data.get('countries', []):
    for city in country.get('cities', []):
        if city.get('uid') == 362:
            for place in city.get('places', []):
                if place.get('spot', False):  # Stationen haben 'spot' == True
                    stations_data.append(place)

print(f"Gesamtanzahl der Stationen: {len(stations_data)}")

# Filter für Stationen mit bike: false
stations_data_bike_false = [station for station in stations_data if not station.get('bike', True)]

# Anzahl der Stationen mit bike: false
total_stations_bike_false = len(stations_data_bike_false)
print(f"📍 Gesamtanzahl der Stationen mit bike: false: {total_stations_bike_false}")

# Terminal-Typen für bike: false extrahieren
terminal_types_bike_false = [station.get('terminal_type', 'unbekannt') for station in stations_data_bike_false]

# Terminal-Typen zählen für bike: false
terminal_type_counts_bike_false = pd.Series(terminal_types_bike_false).value_counts()

print("\n🔢 Terminal-Typen und deren Häufigkeit (bike: false):")
print(terminal_type_counts_bike_false)


In [3]:
import requests
import pandas as pd

# 1. Nextbike API für Berlin abrufen
API_URL = "https://api.nextbike.net/maps/nextbike-live.json?city=362"
response = requests.get(API_URL)
data = response.json()

# 2. Alle Stationen extrahieren (places mit spot == True)
stations_data = []
for country in data.get('countries', []):
    for city in country.get('cities', []):
        if city.get('uid') == 362:
            for place in city.get('places', []):
                if place.get('spot', False):  # Stationen haben 'spot' == True
                    stations_data.append(place)

print(f"Gesamtanzahl der Stationen: {len(stations_data)}")

# 3. Filter für Stationen mit bike: false
stations_data_bike_false = [station for station in stations_data if not station.get('bike', True)]
print(f"📍 Anzahl der Stationen mit bike: false: {len(stations_data_bike_false)}")

# 4. Terminal-Typen für bike: false extrahieren und zählen
terminal_types_bike_false = [station.get('terminal_type', 'unbekannt') for station in stations_data_bike_false]
terminal_type_counts_bike_false = pd.Series(terminal_types_bike_false).value_counts()

print("\n🔢 Terminal-Typen und deren Häufigkeit (bike: false):")
print(terminal_type_counts_bike_false)

# 5. Jelbi-Stationen mit bike: false filtern
jelbi_stations_bike_false = [station for station in stations_data_bike_false if 'Jelbi' in station.get('name', '')]
print(f"\n🚲 Anzahl der Jelbi-Stationen mit bike: false: {len(jelbi_stations_bike_false)}")

# 6. Terminal-Typen für Jelbi-Stationen zählen
jelbi_terminal_types_bike_false = [station.get('terminal_type', 'unbekannt') for station in jelbi_stations_bike_false]
jelbi_terminal_type_counts_bike_false = pd.Series(jelbi_terminal_types_bike_false).value_counts()

print("\n🔢 Terminal-Typen für Jelbi-Stationen mit bike: false:")
print(jelbi_terminal_type_counts_bike_false)

# 7. Optional: Ausgabe der Stationen mit Details (Name, Terminal-Typ)
print("\nBeispielhafte Stationen mit bike: false:")
for station in stations_data_bike_false[:5]:
    print(f"- {station.get('name', 'Unbekannt')} | Terminal-Typ: {station.get('terminal_type', 'unbekannt')}")


Gesamtanzahl der Stationen: 2630
📍 Anzahl der Stationen mit bike: false: 2630

🔢 Terminal-Typen und deren Häufigkeit (bike: false):
free            2487
smart_sign       122
traffic_sign      12
                   5
sign               4
Name: count, dtype: int64

🚲 Anzahl der Jelbi-Stationen mit bike: false: 287

🔢 Terminal-Typen für Jelbi-Stationen mit bike: false:
free            261
traffic_sign     12
smart_sign        7
                  4
sign              3
Name: count, dtype: int64

Beispielhafte Stationen mit bike: false:
- Kurfürstendamm/Rankestraße | Terminal-Typ: free
- EDEKA Schmitt (S Halensee) | Terminal-Typ: free
- Joachim-Friedrich-Str./KuDamm | Terminal-Typ: smart_sign
- Albrecht-Achilles-Straße | Terminal-Typ: free
- Karlsruher Str./Aspria | Terminal-Typ: free
